In [87]:
import pandas as pd
import numpy as np
from data_reader import read_df, interaction_matrix

def user_favorites(df):
    return dict([(user, group.tolist()[::-1]) for user, group in df.groupby('user_id')['item_id']])


df = read_df('test')

fav = user_favorites(df)
users = set(df['user_id'])

# user2index, item2index, inteRaction matrix
u2i, i2i, R = interaction_matrix(df)

gold = { u2i[user]:[i2i[i] for i in items] for user,items in fav.items() }

In [88]:
def recall(gold, sorting):
    best = sorting[:20]
    return len(set(best).intersection(gold)) / len(gold)

def test_sorting():
    user = 'A2G04D4QZAXL15'
    items = fav[user] + fav['A3JT29L4YFEIMJ']
    np.random.shuffle(items)
    i = items[0:5]
    return fav[user], i

def rc_test():
    user, i = test_sorting()
    fav[user], i, recall(fav[user], i)

In [10]:
def dcg(gold, sorting, return_values=False, p=20):
    best = sorting[:p]

    # relevance
    rel = [int(item in gold) for item in best][:p]
    rel = np.array(rel) / np.hstack((1, np.log(2 + np.arange(len(rel)-1),)/np.log(2)))
    
    score = rel.sum()
    return (score, rel.cumsum()) if return_values else score

def ndcg(gold, sorting, return_values=False):
    _, ideal = dcg(gold, gold, return_values=True)
    _, rel = dcg(gold, sorting, return_values=True)
    ndcg = (rel/ideal)

    return ndcg.sum() if not return_values else (ndcg.sum(), ndcg)

def ndcg_test():
    ex = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
    fav_ex = sorted(ex, reverse=True)
    print('DCG', dcg(fav_ex, ex, return_values=True)[1])
    print('NDCG', ndcg(fav_ex, ex, True)[1])
    sc, rel = dcg(*test_sorting(), True)
    return sc, rel

In [61]:
def rank_correlation_fn(gold, rank_fn):
    n = len(gold)

    a = np.arange(n) + 1
    b = np.array([rank_fn(item) for item in gold])

    # spearman
    return 1 - 6*(b - a)**2 / n / (n**2 - 1)

# def test_rank_correlation

def rank_correlation(gold, ranked):
    n = len(gold)

    a = np.arange(n) + 1
    b = np.array(ranked)

    # spearman
    return 1 - (6*(b - a)**2 / n / (n**2 - 1)).sum()

def extract_rank(gold, retrieved):
    n = len(gold)
    return [np.argwhere(gold[i] == retrieved)[0][0]+1 for i in range(n)]



In [12]:
ex = [3, 2, 3, 0, 0, 1, 2, 2, 3, 0]
gold = sorted(ex, reverse=True)

In [27]:
def calc_gamma():
    import numpy as np

    def dm(a):
        a = np.array(a)
        n = len(a)
        A = np.zeros((n, n))
        for i in range(n):
            A[i, :] = a - a[i]
        return A

    A, B = dm(ex), dm(gold)
    gamma = lambda A,B: (A*B).sum() / np.sqrt((A*A).sum() * (B*B).sum())

    return gamma

In [35]:
ts = test_sorting()[1]


In [38]:
st = ts[:]
np.random.shuffle(st)

In [45]:
st = np.array(st)
ts = np.array(ts)

In [64]:
rank_correlation(ts, extract_rank(ts, st))

0.29999999999999993

In [59]:
ts

array(['B0098JU8T2', 'B005HIHD2I', 'B00AE0790U', 'B005TI7NQW',
       'B00HB2JQNM'], dtype='<U10')

In [111]:
import numpy as np

def topk(R, k):
    '''
    returns index of top k items, sorted
    '''

    A = R.argpartition(kth=-k, axis=1)[:, -k:]
    S = np.take_along_axis(R, A, axis=1)
    return np.take_along_axis(A, S.argsort(), axis=1)[:,::-1]

In [120]:
R

array([[0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 0., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       ...,
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.],
       [0., 0., 0., ..., 1., 1., 1.]])

In [121]:
u_len = (R > 0).sum(axis=1)

K = min(u_len.max(), 20)
guess = topk(R, K)

def eval_score(user, fn):
    n = u_len[user]
    retrieved = guess[user, :n]
    gold_n = gold[user][:n]

    return fn(gold_n, retrieved)

fn = recall
scores = [eval_score(user, fn) for user in range(len(users))]

TypeError: 'NoneType' object is not subscriptable

In [122]:
scores

[0.7,
 0.0,
 0.4,
 0.5,
 0.0,
 0.5,
 0.3333333333333333,
 0.3333333333333333,
 0.2857142857142857,
 0.6666666666666666,
 1.0,
 0.3333333333333333,
 0.6,
 0.2,
 1.0,
 0.0,
 0.875,
 0.0,
 0.0,
 0.0,
 0.5,
 0.625,
 1.0,
 0.8,
 0.0,
 0.6666666666666666,
 0.3333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.3333333333333333,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.25,
 0.0,
 0.6666666666666666,
 0.6666666666666666,
 0.6,
 1.0,
 0.0,
 0.5,
 0.3333333333333333,
 0.3333333333333333,
 0.0,
 0.6666666666666666,
 0.25,
 0.3333333333333333,
 0.5,
 0.0,
 0.6,
 0.5,
 1.0,
 0.6666666666666666,
 0.5,
 0.0,
 0.3333333333333333,
 0.6666666666666666,
 0.8181818181818182,
 0.5,
 0.6666666666666666,
 0.0,
 0.2857142857142857,
 0.0,
 1.0,
 0.0,
 0.6,
 0.4,
 0.5555555555555556,
 0.3333333333333333,
 1.0,
 0.3333333333333333,
 0.0,
 0.42857142857142855,
 0.5,
 0.5,
 0.6666666666666666,
 0.0,
 0.3333333333333333,
 0.2,
 0.0,
 0.6666666666666666,
 0.9333333333333333,
 0.625,
 0.5,
 0.0,
 0.5714285714285714,
 0.0,
 0.0,
 0.1

In [86]:
guess

array([[722,  45, 186, ..., 244, 235, 234],
       [722, 246, 245, ..., 237, 234, 235],
       [408, 563, 245, ..., 237, 231, 246],
       ...,
       [453, 245,  44, ..., 237, 232, 246],
       [ 92, 339, 245, ..., 237, 232, 246],
       [158, 246, 245, ..., 237, 234, 235]])

In [74]:
u_len = (R > 0).sum(axis=1)


20

In [108]:
r = np.random.default_rng(1234).random((3,3))
r

array([[0.97669977, 0.38019574, 0.92324623],
       [0.26169242, 0.31909706, 0.11809123],
       [0.24176629, 0.31853393, 0.96407925]])

In [110]:
np.take_along_axis(r, r.argsort(), axis=1)

array([[0.38019574, 0.92324623, 0.97669977],
       [0.11809123, 0.26169242, 0.31909706],
       [0.24176629, 0.31853393, 0.96407925]])